In [1]:
import re
from warnings import simplefilter

import pandas as pd
import pymorphy2
from nltk.corpus import stopwords
from pandas import DataFrame, read_csv, np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

pd.set_option('display.max_columns', 400)
simplefilter(action='ignore', category=FutureWarning)
morph = pymorphy2.MorphAnalyzer()
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

In [7]:
#Regression Analysis
def reg_methods(models, dataset): # Построение регрессионных моделей
    trg = dataset[['likes']]
    trn = dataset.drop(['likes'], axis=1)
    x_trn, x_test, y_trn, y_test = train_test_split(trn, trg, test_size=0.4)
    test_models = DataFrame()
    tmp = {}
    for model in models:
        m = str(model)
        tmp['Model'] = m[:m.index('(')]
        model.fit(list(x_trn.iloc[:,0]),list(y_trn.iloc[:,0]))
        tmp['Reg'] = r2_score(model.predict(list(x_test.iloc[:,0])),list(y_test.iloc[:,0]))
        print(tmp['Reg'])
        test_models = test_models.append([tmp], sort=True)
        if m.__contains__('RandomForestRegressor'):
            get_key(vectorizer.vocabulary_,2940)
            importances = model.feature_importances_
            std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
            indices = np.argsort(importances)[::-1]
            print("Feature ranking:")
            i=0            
            for f in range(len(importances)):
                if i==10:
                    break
                print("%d. feature %s (%f)" % (f + 1, get_key(vectorizer.vocabulary_,[indices[f]]), importances[indices[f]]))
                i+=1
    test_models.set_index('Model', inplace=True, )
    #paint_reg_methods(test_models)

#get words
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

#normalize words
def normalize(text):
    # remove evrything except letters
    new_text = re.sub("[^а-яА-Яa-zA-Z]", " ", text)
    # lower case
    words = new_text.lower().split()
    # remove stop words
    words = [w for w in words if not w in stops]
    # normal form
    words = [morph.parse(w)[0].normal_form for w in words]
    return (words)


In [3]:
#Read dataset
dataset = read_csv('test2.tsv',sep ='\t',error_bad_lines=False)
test = []
data = DataFrame(dataset, columns=['likes', 'coms', 'favs', 'size', 'text'])

In [4]:
#prepare data
res=[]
for item in data['text']:
    res.append(normalize(item))
data['test']=res
vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc,lowercase=False)
X = vectorizer.fit(data['test'])
result=[]
for item in data['test']:
    Y=vectorizer.transform(item)
    result.append(Y.indices)
data['vector']=result
del data['text'], data['coms'], data['favs'], data['size'], data['test']
max = 0
new_vectors = []
for item in data['vector']:
    if len(item) > max:
        max = len(item)
for item in data['vector']:
    if len(item) <= max:
        while len(item)!= max:
            item = np.append(item, -1)
        new_vectors.append(item)
data['new_vector'] = new_vectors
del data['vector']

In [8]:
#main
models = [LinearRegression(),  # метод наименьших квадратов          
          KNeighborsRegressor(n_neighbors=6),  # метод ближайших соседей
          SVR(kernel='linear', max_iter=100000), # метод опорных векторов с линейным ядром
          RandomForestRegressor(n_estimators=100, max_features='sqrt') # случайный лес
          ] #Regression Models
reg_methods(models,data)

-0.008584602326946023
-2.7628225715236594
0.011685977198964692
-4.798437912017825
Feature ranking:
1. feature adventure (0.005981)
2. feature amc (0.004579)
3. feature analog (0.004298)
4. feature action (0.003819)
5. feature accuses (0.003658)
6. feature animemap (0.003484)
7. feature accessibility (0.003462)
8. feature angela (0.003458)
9. feature adaptation (0.003410)
10. feature angelo (0.003280)


c:\users\admin\pycharmprojects\reg_analysis_v2\venv\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
